A simple nn solution with kears

https://www.kaggle.com/knowledgegrappler/a-simple-nn-solution-with-keras-0-48611-pl

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import math

from subprocess import check_output

In [2]:
def rmsle(y, y_pred):
    assert len(y)==len(y_pred)
    to_sum = [(math.log(y_pred[i]+1) - math.log(y[i]+1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/float(len(y)))) ** 0.5

assert文とは

変数が期待する値ではない時に、例外を投げます。

デバッグやテストで使用します。

試行錯誤をしながらのアドホックなデータ分析の前処理等、「とにかく想定と違ったら止める」というだけで良いなら、unittest等のテストクラスを書かなくてよいassert文がお手軽です。

https://qiita.com/nannoki/items/15004992b6bb5637a9cd

In [3]:
dtype = {'train_id':'int32','name':'object','item_condition_id':'int8','category_name':'object','brand_name':'object',
         'price':'float32','shipping':'int8','item_description':'object'}
train = pd.read_csv('/home/kyohei/.kaggle/competitions/mercari-price-suggestion-challenge/train.tsv',
                   delimiter='\t',
                   dtype=dtype,
                   nrows=10000)
test = pd.read_csv('/home/kyohei/.kaggle/competitions/mercari-price-suggestion-challenge/test.tsv',
                   delimiter='\t',
                   dtype=dtype,
                   nrows=10000)

In [4]:
print('Handling missing values...')
def handle_missing(dataset):
    dataset.category_name.fillna(value='missing', inplace=True)
    dataset.brand_name.fillna(value='missing',inplace=True)
    dataset.item_description.fillna(value='missing', inplace=True)
    return(dataset)

train = handle_missing(train)
test = handle_missing(test)
print(train.shape)
print(test.shape)

Handling missing values...
(10000, 8)
(10000, 7)


In [5]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,missing,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,missing,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,missing,44.0,0,Complete with certificate of authenticity


In [6]:
#PROCESS CATEGORICAL DATA
print('Handling categorical variables...')
le = LabelEncoder()

le.fit(np.hstack([train.category_name, test.category_name]))
train.category_name = le.transform(train.category_name)
test.category_name = le.transform(test.category_name)

le.fit(np.hstack([train.brand_name,test.brand_name]))
train.brand_name = le.transform(train.brand_name)
test.brand_name = le.transform(test.brand_name)
del le

train.head(3)

Handling categorical variables...


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,419,1028,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,66,766,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,700,888,10.0,1,Adorable top with a hint of lace and a key hol...


In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
#Process test: raw
print('Text to seq process...')
from keras.preprocessing.text import Tokenizer
raw_text = np.hstack([train.item_description.str.lower(), train.name.str.lower()])

print('Fitting tokenizer...')
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
print('Transforming text to seq...')

train['seq_item_description'] = tok_raw.texts_to_sequences(train.item_description.str.lower())
test['seq_item_description'] = tok_raw.texts_to_sequences(test.item_description.str.lower())
train['seq_name'] = tok_raw.texts_to_sequences(train.name.str.lower())
test['seq_name'] = tok_raw.texts_to_sequences(test.name.str.lower())
train.head(3)

In [ ]:
#dequences variables analysis
max_name_seq = np.max([np.max(train.seq_name.apply(lambda x: len(x))), np.max(test.seq_name.apply(lambda x: len(x)))])
max_seq_item_description = np.max([np.max(train.seq_item_description.apply(lambda x: len(x)))
                                  , np.max(test.seq_item_description.apply(lambda x: len(x)))])
print('max name seq '+str(max_name_seq))
print('max item desc seq '+str(max_seq_item_description))

In [ ]:
train.seq_name.apply(lambda x: len(x)).hist()

In [ ]:
train.seq_item_description.apply(lambda x: len(x)).hist()

In [ ]:
#EMBEDDINGS MAX VALUE
#Base on the histograms, we select the next lengths
MAX_NAME_SEQ = 10
MAX_ITEM_DESC_SEQ = 75
MAX_TEXT = np.max([np.max(train.seq_name.max()),
                   np.max(test.seq_name.max()),
                   np.max(train.seq_item_description.max()),
                   np.max(test.seq_item_description.max())])+2
MAX_CATEGORY = np.max([train.category_name.max(), test.category_name.max()])+1
MAX_BRAND = np.max([train.brand_name.max(), test.brand_name.max()])+1
MAX_CONDITION = np.max([train.item_condition_id.max(), test.item_condition_id.max()])+1

In [ ]:
#Scale target variable
train['target'] = np.log(train.price+1)
target_scaler = MinMaxScaler(feature_range=(-1,1))
train['target'] = target_scaler.fit_transform(train.target.reshape(-1,1))
pd.DataFrame(train.target).hist()

In [ ]:
#extracet development test
dtrain, dvalid = train_test_split(train,random_state=123, train_size=0.99)
print(dtrain.shape)
print(dvalid.shape)

In [ ]:
#keras data definition
from keras.preprocessing.sequence import pad_sequences


def get_keras_data(dataset):
    X = {
        'name':pad_sequences(dataset.seq_name,maxlen=MAX_NAME_SEQ),
        'item_desc':pad_sequences(dataset.seq_item_description,maxlen=MAX_ITEM_DESC_SEQ),
        'brand_name':np.array(dataset.brand_name),
        'category_name':np.array(dataset.category_name),
        'item_condition':np.array(dataset.item_condition_id),
        'num_vars':np.array(dataset[['shipping']])
    }
    
    return X

X_train = get_keras_data(dtrain)
X_valid = get_keras_data(dvalid)
X_test = get_keras_data(test)

コールバックの使い方
https://keras.io/ja/callbacks/

バックエンドとは？
https://keras.io/ja/backend/

In [ ]:
#KERAS MODEL DEFINITION
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K

def get_callbacks(filepath,patience=2):
    es = EarlyStopping('val_loss',patience=patience,mode='min')
    msave = ModelCheckpoint(filepath,save_best_only=True)
    return [es,msave]

def rmsle_cust(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(),None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(),None) + 1.)
    return K.sqrt(K.mean(K.square(first_log - second_log),axis=-1))

def get_model():
    #params
    dr_r = 0.1
    
    #Inputs
    name = Input(shape=[X_train['name'].shape[1]], name='name')
    item_desc = Input(shape=[X_train['item_desc'].shape[1]], name='item_desc')
    brand_name = Input(shape=[1],name='brand_name')
    category_name = Input(shape=[1],name='category_name')
    item_condition = Input(shape=[1],name='item_condition')
    num_vars = Input(shape=[X_train['num_vars'].shape[1]],name='num_vars')
    
    #Embeddings layers
    emb_name = Embedding(MAX_TEXT, 50)(name)
    emb_item_desc = Embedding(MAX_TEXT, 50)(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
    emb_category_name = Embedding(MAX_CATEGORY, 10)(category_name)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    
    #rnn layer
    rnn_layer1 = GRU(16) (emb_item_desc)
    rnn_layer2 = GRU(8) (emb_name)
    
    #main_layer
    main_l = concatenate([
        Flatten() (emb_brand_name)
        , Flatten() (emb_category_name)
        , Flatten() (emb_item_condition)
        , rnn_layer1
        , rnn_layer2
        , num_vars
    ])
    main_l = Dropout(dr_r) (Dense(128) (main_l))
    main_l = Dropout(dr_r) (Dense(64) (main_l))
    
    #output
    output = Dense(1, activation='linear') (main_l)
    
    #mode
    model = Model([name,item_desc,brand_name,category_name,item_condition,num_vars], output)
    model.compile(loss='mse',optimizer='adam',metrics=['mse',rmsle_cust])
    
    return model

model = get_model()
model.summary()